This is a script that uses BioPython and uniprotenator to analyze our sequencing data.

In [34]:
from Bio.Seq import Seq
import string

from urllib2 import urlopen
from lxml import objectify

from Bio import pairwise2

In [5]:
uniprotID = 'P12931'
filename = '../sequencing data by clone/18 - Src GK/SrcGK-T7-Term.seq'

In [26]:
def rev_sequencing_results(filename):
    with open(filename, 'r') as f:
        sequence_rough = f.readlines()[1:]
    sequence_rough = string.join(sequence_rough)
    nucleotide_sequence = sequence_rough.replace('\n ','').replace('\n','')
    seq = Seq(nucleotide_sequence)
    reverse_complement = seq.reverse_complement()
    print ':::FIRST READING FRAME:::'
    print reverse_complement.translate()
    print ':::SECOND READING FRAME:::'
    print reverse_complement[1:].translate()
    print ':::THIRD READING FRAME:::'
    print reverse_complement[2:].translate()

In [27]:
rev_sequencing_results(filename)

:::FIRST READING FRAME:::
TXXPTTEXLXXXGPYADPGTRQGRVGKSPGSRCGWR*SWGRAALERSGWGPGTAPPEXP*RL*SPAPCPRRPSCRKPK**RXSGMRSWFSCTQWCRKSPSTSSLST*ARGASWIS*RERPASTCGCHSSSIWLLRLHPAWPMWRG*TTCTETCGRPTSWWGRTWCARWLTLGWHASSRTTSTQHGKVPSSPSSGQPPRQPSMAGSPSSRMSGPSASC*LS*PPRAGCHTQGWSTGRCWTRWRGATACPARPSAPSRCMTSCASAGGRNLRSGPLLSTCRPSWRTTSPRQSPSTSLERTYRDPNSSSVDKXXAALEHHHHHH*DPAANKXXXXXXXX
:::SECOND READING FRAME:::
RXSQRPKXXXXXXHMQTQGLAKDAXGNPPGVAAAGGEAGAGLLWRGLDGDLERHHQSXHKDSEARHHVPGGLPAGSPSDEEXPA*EAGSAVRSGVGRAHLHRH*VHEQGEPPGFPEGRDRQVPAAATARRYGCSDCIRHGLCGEDELRAPRPAGGQHPGGGEPGVQGG*LWAGTPHRGQRVHSTARCQVPHQVDSPRGSPLWPVHHQVGCLVLRHPAD*ADHQGPGAIPRDGQQGGAGPGGEGLPHALPARVPRVAA*PHVPVLAEGT*GAAHF*VPAGLPGGLLHLDRAPVPAWREPIGIRIRAPSTSXRPHSSTTTTTTEIRLLTKXEXXXXXX
:::THIRD READING FRAME:::
DXPNDRXPXLXRXICRPRDSPRTRXEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVXIKTLKPGTMSPEAFLQEAQVMKKXRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMP

In [28]:
def rev_sequencing_pick_frame(filename,frame):
    with open(filename, 'r') as f:
        sequence_rough = f.readlines()[1:]
    sequence_rough = string.join(sequence_rough)
    nucleotide_sequence = sequence_rough.replace('\n ','').replace('\n','')
    seq = Seq(nucleotide_sequence)
    reverse_complement = seq.reverse_complement()
    sequencing_result = str(reverse_complement[frame:].translate())
    
    return sequencing_result

In [29]:
sequencing_result = rev_sequencing_pick_frame(filename,2)

In [30]:
sequencing_result

'DXPNDRXPXLXRXICRPRDSPRTRXEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVXIKTLKPGTMSPEAFLQEAQVMKKXRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL*GSEFELRRQAXGRTRAPPPPPLRSGC*QXXXXXXXX'

In [42]:
def uniprot_sequence(uniprotID):

    # Define url from which to download xml.
    url='http://www.uniprot.org/uniprot/'
    r1 = urlopen(url+'%s' % uniprotID +'.xml')

    # Read xml file.
    my_xml = r1.read()
    # Take out annoying head that makes xml file difficult to parse.
    my_xml = my_xml.replace('<uniprot xmlns="http://uniprot.org/uniprot" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://uniprot.org/uniprot http://www.uniprot.org/support/docs/uniprot.xsd">','<uniprot>', 1)

    # Write xml file.
    with open("%s.xml" % uniprotID,"w") as file:
        file.write(my_xml)

    # Define uniprot sequence from xml file:
    root = objectify.fromstring(my_xml)
    uniprot_sequence = [element.text for element in root.findall('.//sequence')]
    
    return uniprot_sequence

In [43]:
uniprot_sequence = uniprot_sequence(uniprotID)

In [44]:
uniprot_sequence[2]

'\nMGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAE\nPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGD\nWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRES\nETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGL\nCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTL\nKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKY\nLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYT\nARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVER\nGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL\n'

In [45]:
uniprot_sequence = uniprot_sequence[2].replace('\n','')

In [46]:
uniprot_sequence

'MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL'

In [47]:
alignments = pairwise2.align.globalxx(sequencing_result, uniprot_sequence)
alignments[0]

('---------D-------X-P--N---------------------D--R------------------------------X-P-------------------X-L----X-R-------------------------------------------X-I----C-R-------P------------------------------------R--D-------SPRT---------------------R------------------X-EIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVXIKTLKPGTMSPEAFLQEAQVMKKXRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL*GSEFELRRQAXGRTRAPPPPPLRSGC*QXXXXXXXX',
 'MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITS-RTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNE

Cool! What about our forward sequencing data?

In [48]:
filename = '../sequencing data by clone/18 - Src GK/SrcGK-T7.seq'

In [49]:
def for_sequencing_results(filename):
    with open(filename, 'r') as f:
        sequence_rough = f.readlines()[1:]
    sequence_rough = string.join(sequence_rough)
    nucleotide_sequence = sequence_rough.replace('\n ','').replace('\n','')
    seq = Seq(nucleotide_sequence)
    reverse_complement = seq.reverse_complement()
    print ':::FIRST READING FRAME:::'
    print seq.translate()
    print ':::SECOND READING FRAME:::'
    print seq[1:].translate()
    print ':::THIRD READING FRAME:::'
    print seq[2:].translate()

In [50]:
for_sequencing_results(filename)

:::FIRST READING FRAME:::
XXXXXXXX*NNFV*L*EGDIPWAAAIIIIITITISQRPKTFTSRAICRPRDSPRTRGKSPGSRCGWR*SWGRAALERSGWGPGTAPPEWP*RL*SPAPCPRRPSCRKPK**RSSGMRSWFSCTQWCRKSPSTSSLST*ARGASWIS*RERPASTCGCHSSSIWLLRLHPAWPMWRG*TTCTETCGRPTSWWGRTWCARWLTLGWHASSRTTSTQHGKVPSSPSSGQPPRQPSMAGSPSSRMSGPSASC*LS*PPRAGCHTQGWSTGRCWTRWRGATACPARPXAPSRCMTSCASAGGRNLRSGPLLSTCRXSWRTTSSRQXPVPAXXXL*GSXXELRRXLAAALEHXXHXHX
:::SECOND READING FRAME:::
XXXXXXXXRIILFNFKKEIYHGQQPSSSSSRLRYPNDRKPLLPGPYADPGTRQGRVGNPPGVAAAGGEAGAGLLWRGLDGDLERHHQSGHKDSEARHHVPGGLPAGSPSDEEAPA*EAGSAVRSGVGRAHLHRH*VHEQGEPPGFPEGRDRQVPAAATARRYGCSDCIRHGLCGEDELRAPRPAGGQHPGGGEPGVQGG*LWAGTPHRGQRVHSTARCQVPHQVDSPRGSPLWPVHHQVGCLVLRHPAD*ADHQGPGAIPRDGQQGGAGPGGEGLPHALPARXPRVAA*PHVPVLAEGT*GAAHF*VPAGLPGGLLHLDRXQYQPXRXYRXPXSSSVXSLXPHSSTXXTXT
:::THIRD READING FRAME:::
XXXXXXXLE*FCLTLRRRYTMGSSHHHHHHDYDIPTTENLYFQGHMQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQ

In [51]:
def for_sequencing_pick_frame(filename,frame):
    with open(filename, 'r') as f:
        sequence_rough = f.readlines()[1:]
    sequence_rough = string.join(sequence_rough)
    nucleotide_sequence = sequence_rough.replace('\n ','').replace('\n','')
    seq = Seq(nucleotide_sequence)
    reverse_complement = seq.reverse_complement()
    sequencing_result = str(seq[frame:].translate())
    
    return sequencing_result

In [53]:
sequencing_result = for_sequencing_pick_frame(filename,2)

In [54]:
sequencing_result

'XXXXXXXLE*FCLTLRRRYTMGSSHHHHHHDYDIPTTENLYFQGHMQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPXCPESLHDLMCQCWRKEPEERPTFEYLQXFLEDYFISTXPSTSLXEXIGXXXRAPSXACXRTRAPXXPPL'

In [55]:
alignments = pairwise2.align.globalxx(sequencing_result, uniprot_sequence)
alignments[0]

('------------X-XXXXXXL-----------------------------------------E----*-F------------C-L-----T---L-----R-----------R-----RYTMG--------S--------S---HHHHHHD-------Y-D-I--------------P--T--------T---------------E-N--------L-----Y------F----Q----------G--H---------M-QTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVIEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPXCPESLHDLMCQCWRKEPEERPTFEYLQXFLEDYFISTXPSTSLXEXIGXXXRAPSXACXRTRAP-XXPPL',
 'MGSNKSKPKDASQRR-R-S-LEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNN-TEGDWWLAHSLSTGQTGYIPSNYVA-P-S-DSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYT

Is there a way to get this analysis to tell us what the uniprot stop and start numbers are and what the linker and HisTag are?

NEXT: Make same function for forward and reverse and put it into a module in PlasmidLibraryTools/PlasmidLibraryTools/ directory!